Importing Libraries

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import albumentations as A

Augmentation Function

In [ ]:
# Same as in main pipeline
def get_training_augmentation():
    train_transform = [
        # A.Resize(height=352, width=768),  # ✅ safe size
        A.HorizontalFlip(p=0.5),

        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        # A.PadIfNeeded(min_height=320, min_width=320, border_mode=0),
        # A.RandomCrop(height=320, width=320),

        A.GaussNoise(p=0.2),
        A.Perspective(p=0.5),

        A.OneOf(
            [
                A.CLAHE(p=1),
                A.RandomBrightnessContrast(p=1),  # ✅ updated line
                A.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.Sharpen(p=1),
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),

        A.Lambda(mask=round_clip_0_1)
    ]

    return A.Compose(train_transform)

In [ ]:
def get_variable_aug_pipeline():
    return A.Compose([
        A.OneOf([
            A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1),
            A.RandomRotate90(p=1),
        ], p=0.8),

        A.OneOf([
            A.ShiftScaleRotate(scale_limit=0.2, rotate_limit=20, shift_limit=0.1, p=1, border_mode=0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1),
            A.GridDistortion(p=1),
            A.OpticalDistortion(p=1),
        ], p=0.7),

        A.OneOf([
            A.CLAHE(p=1),
            A.RandomBrightnessContrast(p=1),
            A.RandomGamma(p=1),
            A.HueSaturationValue(p=1),
            A.RGBShift(p=1),
        ], p=0.8),

        A.OneOf([
            A.GaussNoise(var_limit=(10.0, 50.0), p=1),
            A.Blur(blur_limit=3, p=1),
            A.MotionBlur(blur_limit=3, p=1),
            A.MedianBlur(blur_limit=3, p=1),
        ], p=0.5),

        A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.2),
    ])


Augmentation

In [ ]:
input_img_dir = 'images_3/images_train/img'
input_mask_dir = 'images_3/images_all/mask'
output_img_dir = 'images_3/augmented/img'
output_mask_dir = 'images_3/augmented/mask'
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

In [ ]:
# Parameters
N_AUGS = 5
augment = get_training_augmentation()

# CHANGE DEPENDING ON WHICH ONE WE WANT TO USE !!!!!!
# N_AUGS = 10
# augment = get_variable_aug_pipeline()

# Loop through all input images
for fname in tqdm(os.listdir(input_img_dir)):
    if not fname.endswith(('.jpg', '.png', '.jpeg')):
        continue

    img_path = os.path.join(input_img_dir, fname)
    mask_path = os.path.join(input_mask_dir, fname)

    image = cv2.imread(img_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    for i in range(N_AUGS):
        augmented = augment(image=image, mask=mask)
        aug_img = augmented['image']
        aug_mask = augmented['mask']

        base = os.path.splitext(fname)[0]
        img_out = os.path.join(output_img_dir, f"{base}_aug{i}.png")
        mask_out = os.path.join(output_mask_dir, f"{base}_aug{i}.png")

        cv2.imwrite(img_out, aug_img)
        cv2.imwrite(mask_out, aug_mask)